In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from collections import OrderedDict

from astropy.table import Table, vstack, Column

from astropy.io import fits
import numpy as np

from os import listdir
from os.path import isfile, join



# Configuration

In [2]:
input_catalogs_path ='/home/aalvarez/Work/Data/SC7/TU/Stars/Deep/HealPix/Deep.fits'
output_catalog_path ='/home/aalvarez/Phosphoros/Catalogs/SC7_Stars/'

In [3]:
# filter names
filters_name = ["TU_FNU_VIS", "TU_FNU_Y_NISP", "TU_FNU_J_NISP", "TU_FNU_H_NISP", 
        "TU_FNU_G_DECAM", "TU_FNU_R_DECAM", "TU_FNU_I_DECAM", "TU_FNU_Z_DECAM", 
        "TU_FNU_U_MEGACAM", "TU_FNU_R_MEGACAM", "TU_FNU_G_JPCAM", 
        "TU_FNU_I_PANSTARRS", "TU_FNU_Z_PANSTARRS", "TU_FNU_Z_HSC", "TU_FNU_U_LSST", 
        "TU_FNU_G_LSST", "TU_FNU_R_LSST", "TU_FNU_I_LSST", "TU_FNU_Z_LSST"]

# 10-sigma depths in Euclid-survey bands
# from J.-C. Cuillandre's report
depths = {
        "TU_FNU_VIS":24.5, "TU_FNU_Y_NISP":24.0 , "TU_FNU_J_NISP":24.0, "TU_FNU_H_NISP":24.0 , 
        "TU_FNU_G_DECAM":24.6, "TU_FNU_R_DECAM":24.4, "TU_FNU_I_DECAM":23.7, "TU_FNU_Z_DECAM":23.0, 
        "TU_FNU_U_MEGACAM":23.6, "TU_FNU_R_MEGACAM":23.9, "TU_FNU_G_JPCAM":24.5, 
        "TU_FNU_I_PANSTARRS":23.6, "TU_FNU_Z_PANSTARRS":23.4, "TU_FNU_Z_HSC":23.4, "TU_FNU_U_LSST":24.2, 
        "TU_FNU_G_LSST":24.5, "TU_FNU_R_LSST":23.9, "TU_FNU_I_LSST":23.6, "TU_FNU_Z_LSST":23.4
        }

# training should be 5 times deeper (25x exposure time)
depths_deep = {}
for d in depths:
    depths_deep[d] = depths[d]+2.5*np.log10(5.0)


# sky brightness in Euclid-survey bands
sky_AB={
        "TU_FNU_VIS":22.33, "TU_FNU_Y_NISP":22.10 , "TU_FNU_J_NISP":22.11, "TU_FNU_H_NISP":22.28 , 
        "TU_FNU_G_DECAM":22.00, "TU_FNU_R_DECAM":20.80, "TU_FNU_I_DECAM":20.30, "TU_FNU_Z_DECAM":19.40, 
        "TU_FNU_U_MEGACAM": 22.70, "TU_FNU_R_MEGACAM":20.80, "TU_FNU_G_JPCAM":22.00, 
        "TU_FNU_I_PANSTARRS":20.30, "TU_FNU_Z_PANSTARRS":19.40, "TU_FNU_Z_HSC":23.4, "TU_FNU_U_LSST": 22.70, 
        "TU_FNU_G_LSST":22.00, "TU_FNU_R_LSST":20.80, "TU_FNU_I_LSST":20.30, "TU_FNU_Z_LSST":19.40
        }

In [4]:
catalog_output_type="deep" # "deep" or "wide"
cuttof_VIS_MAB = 24.0

# Functions


In [11]:
def simulate(emulated, filters_name, depths, sky_AB):

    # create table
    t = Table()

    # first copy all non-photometry columns
    for c in emulated.columns:
        if c not in filters_name:
            t[c] = emulated[c]

    # loop over filters
    for f in filters_name:
        # compute unique flux error per band from background rms
        flux_lim = pow(10.0, (-depths[f]+23.9)/2.5)

        flux_err = compute_err(
            emulated[f], flux_lim/10.0, sky_AB[f])
        flux = emulated[f] + np.random.normal(0.0, flux_err)
        mag, mag_err = compute_mags(flux, flux_err, flux_lim, emulated[f])

        t[f+'_obs'] = Column(flux, unit='uJy', dtype=np.float32)
        t[f+'_obs_err'] = Column(flux_err, unit='uJy', dtype=np.float32)
        t[f+'_obs_mag'] = Column(mag, unit='mag', dtype=np.float32)
        t[f+'_obs_mag_err'] = Column(mag_err, unit='mag', dtype=np.float32)

    return t

def compute_err(f, f_bgk_err_typical, sky_AB):
    """ Compute the total flux error from the
    sky background and flux photon count

    OUPUT:
    - flux error
    """
    r = 3

    # background flux error adjusted to
    # the galaxy aperture
    f_err_bgk = f_bgk_err_typical

    # background flux adjusted to
    # the galaxy aperture
    f_bgk = AB_to_uJy(sky_AB)*np.pi*r**2

    # galaxy photon error
    f_err = f_err_bgk*np.sqrt(f/f_bgk)

    # total error
    return np.sqrt(f_err_bgk**2+f_err**2)


def AB_to_uJy(AB):
    return pow(10.0, -0.4*(AB - 23.9))

def uJy_to_AB(muJy):
    return -2.5*np.log10(muJy)+23.9

def compute_mags(f, fErr, flux_limit, fTrue):
    """Compute mags and mag errors from flux
    and flux errors

    When the flux is negative or close to
    1-sigma detection, put -1.
    """

    N = len(f)
    mag = np.zeros(N)-1.0
    magErr = np.zeros(N)-1.0

    # detected = (f > flux_lim/50.0) & (f > 0.0) & (fTrue > 0.0)
    detected = (f > 0.0) & (fTrue > 0.0)
    mag[detected] = -2.5*np.log10(f[detected])+23.9
    magErr[detected] = 1.0857*fErr[detected]/fTrue[detected]

    # mag = uJy_to_AB(f)
    # magErr = 1.0857*fErr/f

    return mag, magErr



In [6]:
def openTable(file_path):
    print('Processing file :' + file_path)
    hdul = fits.open(file_path)
    return Table(hdul[1].data)
    

In [7]:
def filterTableMag(table, mag_cutoff):
    mag_vis = table['TU_FNU_VIS_obs_mag']
    mask = mag_vis <= mag_cutoff
    return table[mask]

# Processing

In [16]:
if catalog_output_type == 'deep':
    d = depths_deep
else:
    d = depths


emulated = openTable(input_catalogs_path)
table = simulate(emulated, filters_name, d, sky_AB)
del emulated
filtered_table = filterTableMag(table, cuttof_VIS_MAB)
del table

# https://astronomy.swin.edu.au/cosmos/i/Interstellar+Reddening
# AV = 3.2E(B-V)
# AV/3.2 = E(B-V)
filtered_table.add_column(Column(filtered_table['AV']/3.2, name='ebv'))

filtered_table.write(output_catalog_path + os.path.basename(input_catalogs_path), format='fits', overwrite=True)


Processing file :/home/aalvarez/Work/Data/SC7/TU/Stars/Deep/HealPix/Deep.fits
